# Final Project

-   ncyc - מספר סידורי של מספר המחזור
-   time - זמן בשנים
-   Mwd - מסת הננס הלבן ביחידות של מסות שמש
-   Ms - מסת הכוכב המשני ביחידות של מסות שמש
-   Mdot - קצב הספיחה ביחידות של מסות שמש לשנים
-   a - המרחק בין מרכזי המסות של שני הגופים ביחידות של רדיוסי שמש
-   Porb(hr) - זמן מחזור סיבוב אחד ביחידות של שעות
-   Lirr - בהירות מוקרנת של הכוכב המשני (אנרגיה ליחדות זמן) ביחידות של בהירות שמש
-   Ls - בהירות הכוכב המשני
-   LRD - בהירות משוקללת של הכוכב המשני
-   TeffWD - טמפרטורה אפקטיבית של הננס הלבן בקלווין
-   LWD - בהירות הננס הלבן
-   Lacc - בהירות ספיחה
-   Mboltot - מגניטודה בולומטרית כוללת (מתקשר לבהירות)
-   Rwd - רדיוס הננס הלבן
-   trec - זמן מחזור נובה של המחזור הקודם ביחידות של שנים

## Checklist for File Cleaning and Analysis

### File Cleaning

-   [x] **Delete the first cycle** from each file (`ncyc`), exept the first file.
-   [ ] **Connect the files**:

    -   [x] Rearrange the numbering of the cycles to start from `1` and continue sequentially to the end of the data from the second file.
    -   [ ] Rearrange the `time` column to continue seamlessly:
        -   Adjust so that time does not restart with the new file but continues from the previous file.

-   [ ] **Support for multiple files**:
    -   [ ] Ensure functionality works for more than two files, with seamless handling of cycle numbering and time adjustment.

### Data Analysis

-   [ ] **Output parameters for each nova cycle**:
    -   [ ] Calculate and output the following for the **adsorption rate (Mdot)**:
        -   [ ] Initial value
        -   [ ] Final value
        -   [ ] Minimum value
        -   [ ] Average value
    -   [ ] Create a table showing these four parameters for each cycle number.

### Future Steps

-   [ ] Extend parameter outputs:
    -   [ ] Add more parameters to the table.
    -   [ ] Include both directly retrieved and calculated values.
----

In [6]:
import os
import pandas as pd
# import numpy as np
# import argparse

In [7]:
# # Create the parser
# parser = argparse.ArgumentParser(description="Process some arguments.")

# # Add arguments
# parser.add_argument('--input', type=str, help="Input file path")
# parser.add_argument('--output', type=str, help="Output file path")
# parser.add_argument('--verbose', action='store_true', help="Enable verbose mode")

# # Parse arguments
# args = parser.parse_args()

# print("Input:", args.input)
# print("Output:", args.output)
# print("Verbose Mode:", args.verbose)

In [8]:
# Directory containing the files
data_folder = "data"

# Get a list of all files in the folder
file_list = os.listdir(data_folder)

# Create a dictionary to store DataFrames
dataframes = {}

# Step 1: Load the files into the dictionary
for file_name in file_list:
    file_path = os.path.join(data_folder, file_name)
    
    try:
        df = pd.read_csv(file_path, sep=r'\s+')  # Load each file into a DataFrame
        
        # Convert all columns to numeric where possible
        for col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce') # Convert to float64 or NaN
        
        dataframes[file_name] = df  # Store it in the dictionary
        print(f"Loaded {file_name} successfully.")
    except Exception as e:
        print(f"Failed to load {file_name}: {e}")

Loaded MdotfromMRATE_A successfully.
Loaded MdotfromMRATE_B successfully.
Loaded MdotfromMRATE_C_COPY_OF_A successfully.


In [ ]:
# Step 2 & 3: Filter, adjust 'ncyc', and combine into one DataFrame
combined_df = pd.DataFrame()  # Initialize an empty DataFrame
current_max_ncyc = 0  # Track the current maximum value of 'ncyc'
last_time_value = 0

for i, (file_name, df) in enumerate(dataframes.items()):
    if "ncyc" in df.columns:  # Ensure the 'ncyc' column exists
        # Explicitly make a copy to avoid SettingWithCopyWarning
        df = df.copy()

        df["ncyc"] = pd.to_numeric(df["ncyc"], errors="coerce")  # Ensure numeric type
        
        # Filter rows if it's not the first file
        if i != 0:
            df = df[df["ncyc"] != 1]
        
        # Adjust 'ncyc' values to continue from the previous max
        df["ncyc"] += current_max_ncyc
        current_max_ncyc = df["ncyc"].max() -1  # Update the max value for the next DataFrame

        # Adjust 'time' values to continue from the last value
        df['time'] += last_time_value
        last_time_value = df["time"].iloc[-1]

        # Combine the DataFrame
        combined_df = pd.concat([combined_df, df], ignore_index=True)
        print(f"Processed and added {file_name} to the combined DataFrame.")
    else:
        print(f"'ncyc' column not found in {file_name}. Skipping.")

print("All DataFrames filtered, adjusted, and combined successfully.")


Processed and added MdotfromMRATE_A to the combined DataFrame.
Processed and added MdotfromMRATE_B to the combined DataFrame.
Processed and added MdotfromMRATE_C_COPY_OF_A to the combined DataFrame.
All DataFrames filtered, adjusted, and combined successfully.


In [13]:
last_row = dataframes['MdotfromMRATE_A'].shape[0]
subset = combined_df.loc[last_row - 10: last_row + 10]
print(f'{last_row = }')
subset
working_columns = ['ncyc', 'time', 'Mwd', 'Ms', 'Mdot']
test_sub = subset[working_columns]
test_sub.shape
test_sub['new_col'] = test_sub['Mwd'] - 1
test_sub
combined_df.tail(10)

last_row = 19298


C:\Users\jenya\AppData\Local\Temp\ipykernel_18688\881003370.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_sub['new_col'] = test_sub['Mwd'] - 1


,ncyc,time,Mwd,Ms,Mdot,a,Porb(hrs),Lirr,Ls,LRD,TeffWD,LWD,Lacc,Mboltot,Rwd,trec
85463,375,8.982067e+09,0.443262,0.196792,4.802966e+44,0.685440,1.972652,357.18647,0.002878,357.18647,818.546851,307.490716,1.143503e+48,-115.425593,873.630644,2.570436e+07
85464,375,8.982067e+09,0.443262,0.196792,4.802966e+44,0.685443,1.972664,357.18647,0.002878,357.18647,831.722773,358.637915,1.093189e+48,-115.376738,913.839347,2.570436e+07
85465,375,8.982067e+09,0.443262,0.196792,4.802966e+44,0.685445,1.972672,357.18647,0.002878,357.18647,842.251159,408.261394,1.050704e+48,-115.333701,950.790560,2.570436e+07
85466,375,8.982067e+09,0.443262,0.196792,4.802966e+44,0.685448,1.972685,357.18647,0.002878,357.18647,850.889005,456.489596,1.014137e+48,-115.295241,985.073321,2.570436e+07
85467,375,8.982067e+09,0.443262,0.196792,4.802966e+44,0.685450,1.972693,357.18647,0.002878,357.18647,858.117774,503.426954,9.821814e+47,-115.260479,1017.122767,2.570436e+07
85468,375,8.982067e+09,0.443262,0.196792,4.802966e+44,0.685453,1.972706,357.18647,0.002878,357.18647,864.271369,549.187183,9.539073e+47,-115.228765,1047.270530,2.570436e+07
85469,375,8.982067e+09,0.443262,0.196792,4.802966e+44,0.685455,1.972714,357.18647,0.002878,357.18647,869.584242,593.871303,9.286312e+47,-115.199608,1075.775840,2.570436e+07
85470,375,8.982067e+09,0.443262,0.196792,4.802966e+44,0.685458,1.972727,357.18647,0.002878,357.18647,874.225733,637.567135,9.058376e+47,-115.172626,1102.845633,2.570436e+07
85471,375,8.982067e+09,0.443262,0.196792,4.802966e+44,0.685460,1.972735,357.18647,0.002878,357.18647,878.319976,680.346606,8.851289e+47,-115.147516,1128.648036,2.570436e+07
85472,375,8.982067e+09,0.443262,0.196792,4.802966e+44,0.685463,1.972747,357.18647,0.002878,357.18647,881.963617,722.280265,8.661929e+47,-115.124037,1153.321728,2.570436e+07


- [x] add pre line time to the new added file.
- [x] group tables by `ncyc` and find min, max, mean of the `Mdot`.
- [x] dtime = last - first
- [x] mean = sum([mdot * dtime]) / sum(dtime)
- [x] create new table with one ncyc. min mdot, max mdot, mean mdot, first mdot, last mdot, and add time.

In [20]:
pick_first_time = True  # Set to False to pick last time instead

if pick_first_time:
    time_summary = combined_df.groupby("ncyc")["time"].first().reset_index(name="time")
else:
    time_summary = combined_df.groupby("ncyc")["time"].last().reset_index(name="time")

summary = combined_df.groupby("ncyc")["Mdot"].agg(["min", "max"]).reset_index()

combined_df["dtime"] = combined_df["time"].diff()
# Compute mean using the weighted method
combined_df["weighted_mdot"] = combined_df["Mdot"] * combined_df["dtime"]

weighted_mean = combined_df.groupby("ncyc").apply(lambda x: x["weighted_mdot"].sum() / x["dtime"].sum() if x["dtime"].sum() != 0 else 0).reset_index(name="mean")

# Merge results
summary = summary.merge(weighted_mean, on="ncyc", how="left")
summary = summary.merge(time_summary, on="ncyc", how="left")
summary

C:\Users\jenya\AppData\Local\Temp\ipykernel_18688\2511163371.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_mean = combined_df.groupby("ncyc").apply(lambda x: x["weighted_mdot"].sum() / x["dtime"].sum() if x["dtime"].sum() != 0 else 0).reset_index(name="mean")


,ncyc,min,max,mean,time
0,1,4.964646e-16,9.108254e-11,6.904970e-11,6.379767e+03
1,2,5.331841e-45,9.104004e-11,3.040312e-11,7.924395e+06
2,3,8.855230e-60,9.087595e-11,2.509398e-11,2.967372e+07
3,4,2.001980e-63,9.064199e-11,2.364955e-11,5.826328e+07
4,5,8.850385e-66,9.042900e-11,2.344314e-11,8.860343e+07
...,...,...,...,...,...
370,371,1.225799e-75,7.753083e-11,2.320735e-11,8.877855e+09
371,372,8.884983e-78,7.765424e-11,2.312616e-11,8.903371e+09
372,373,1.974622e-76,7.777962e-11,2.939927e-11,8.929198e+09
373,374,3.905939e-78,7.790694e-11,2.325222e-11,8.956048e+09
